In [29]:
import pandas as pd
import numpy as np
import os
import random

import warnings

warnings.filterwarnings(action='ignore')

k = 100 # k개까지만 문제 기록을 남기려고함.

path = '/opt/ml/input/data/'

train = pd.read_csv(path + 'train_data.csv')
test = pd.read_csv(path + 'test_data.csv')
dat = pd.concat([train, test])
dat = dat.loc[::-1]
dat.reset_index(drop= True, inplace= True)

dat['tem'] = 1
dat['seq'] = dat.groupby('userID')['tem'].apply(lambda x : x.cumsum())

In [30]:
# test 데이터 만들기.
test_id = test['userID'].unique()
test = dat[(dat['userID'].isin(test_id)) & (dat['seq'] <= k)]
train = dat[~((dat['userID'].isin(test_id)) & (dat['seq'] == 1))]
train['seq'] = train.apply(lambda x : x['seq'] - 1  if x['userID'] in test_id else x['seq'], axis = 1)

# valid 데이터 만들기.
valid_data = train[train['seq'] <= k]

# 뒷처리부분.
user_num = dat['userID'].nunique()
train_new = train.copy()

# seq 하나씩 내리고 0된 값 처리하기.
train_new['seq'] = train_new['seq'] - 1
train_new = train_new[train_new['seq'] >= 1]

# train_new 데이터에 문제 너무 적게 푼 유저 정리하기.
tem = train_new.drop_duplicates(['userID'], keep='last')[['userID','seq']]
lst = list(tem[tem['seq'] < 14]['userID'])
train_new = train_new[~train_new['userID'].isin(lst)]

# 첫번째 train_datas 만들기.
train_datas = train_new[train_new['seq'] <= k]

In [31]:
for i in range(1700):
    # seq 하나씩 내리고 0된 값 처리하기.
    train_new['seq'] = train_new['seq'] - 1
    train_new = train_new[train_new['seq'] >= 1]
    
    # 문제를 적게 푼 유저 정리하기.
    tem = train_new.drop_duplicates(['userID'], keep='last')[['userID','seq']]
    lst = list(tem[tem['seq'] < 14]['userID'])
    train_new = train_new[~train_new['userID'].isin(lst)]
    train_new['userID'] = train_new['userID'] + user_num
    
    # train_data 만들기.
    train_data = train_new[train_new['seq'] <= k]
    train_datas = pd.concat([train_datas, train_data])

    if i % 50 == 0:
        print(i)

0
50
100
150


In [ ]:
valid_data['userID'].value_counts()[-10:]

7349    14
6981    14
7330    14
7084    14
7422    14
7417    14
7390    13
7252    13
7414    13
7441     9
Name: userID, dtype: int64

In [ ]:
train_datas['userID'].value_counts()

625982    100
834174    100
834203    100
834202    100
834201    100
         ... 
512203     14
512428     14
512586     14
154997     14
7439       14
Name: userID, Length: 1039956, dtype: int64

In [ ]:
train_datas

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,tem,seq
2,7439,A040130003,A040000130,1,2020-10-14 23:08:02,8244,1,1
3,7439,A040130002,A040000130,1,2020-10-14 23:07:41,8832,1,2
4,7439,A040130001,A040000130,0,2020-10-14 23:07:23,8832,1,3
5,7439,A040197006,A040000197,1,2020-08-21 07:39:45,2132,1,4
6,7439,A040197005,A040000197,0,2020-08-21 07:39:40,2132,1,5
...,...,...,...,...,...,...,...,...
2526251,1488400,A080072003,A080000072,0,2020-08-26 22:10:03,4974,1,96
2526252,1488400,A080072002,A080000072,0,2020-08-26 22:10:00,4974,1,97
2526253,1488400,A080072001,A080000072,0,2020-08-26 22:09:41,4974,1,98
2526254,1488400,A080070008,A080000070,1,2020-08-25 23:25:40,2887,1,99
